In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pandas as pd
from scipy.interpolate import CubicSpline as cs
from matplotlib.pyplot import contour as con

In [ ]:
field = pd.read_csv('C://MADX//output_map.txt',sep = " ")
print(field)

In [ ]:
plt.figure()
field.plot()

In [ ]:
x_old = field.iloc[0:113,0]
y_old = field.columns.astype(float)[1:]
B_old = field.iloc[0:113,1:32]
print(B_old)
y,x = np.meshgrid(y_old,x_old)
levels = np.linspace(-5000,5000,2000)
con(y_old,x_old,B_old,levels = levels)

In [ ]:
fig = plt.figure()
ax3D = fig.add_subplot(111, projection='3d')
print(x.shape,y.shape,B_old.shape)
print(B_old)
from matplotlib import cm
ax3D.plot_surface(x,y,B_old,cmap=cm.coolwarm,linewidth=0, antialiased=False)

In [ ]:
import plotly
import plotly.graph_objects as go
fig = go.Figure(data=[go.Surface(x=y_old,y=x_old,z=B_old)])
fig.update_layout(title='Field Distribution', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))



In [ ]:
## Raw magnetic field map interpolation
from scipy.interpolate import Rbf
y,x = np.meshgrid(y_old,x_old)
spline = Rbf(y,x,B_old)
x = np.linspace(x_old[0],x_old[112],449)
print(x)
y = np.linspace(y_old[0],y_old[30],93)
print(y)
y1,x1 = np.meshgrid(y,x)
B_new = spline(y1,x1)
print(B_new)
fig1 = go.Figure(data=[go.Surface(x=x1,y=y1,z=B_new)])
fig1.update_layout(title='Interpolated Field Distribution', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

In [ ]:
## Particle parameters
E = 3.005668*10**9 # Calculated energy = B*R*c [eV]
mc = 0.5109989*10**6 # Rest mass [eV]
m = 9.10938E-31 # Electron mass [kg] 
c = 299792458 # Light speed [m/s]
e = 1.6021766208*10**(-19) # Elementary charge [C]
gamma = E/mc # Lorentz factor 
Vx0 = (1-1/gamma**2)**(1/2)*c # Initial velocity Vx [m/s]
V_full = (1-1/gamma**2)**(1/2)*c
Vy0 = 0 # Initial velocity Vy [m/s]
R = 8.876742532 # Bending radius [m]
#B = 1.12946 # Magnetic field [T]
B = 1.12945986
alpha_req = 8.3911 # Required bending angle [deegres]
#E0 = B*R*c # Calculated Energy [eV]
k = 0
for i in range(0, len(y)):
    if y[i] == 0.0:
        print(i,y[i])
        break
y0 = i
for i in range(0, len(x)):
    if x[i] == 0.0:
        print(i,x[i])
        break
x0 = i

print(B_new[x0,y0])    
print(spline(y[y0],x[x0]))
print(y[y0],x[x0])

In [ ]:
## Runge-Kutta numerical algorithm
def rungeKutta(x0,y0,Vx0,Vy0,q):
    dt = 10**(-13)
    
    x1 = x0 + Vx0*dt
    y1 = y0 + Vy0*dt
    
    const = q/(gamma*m)/10000
    B0 = spline(100*y0,100*x0)*const
    B1 = spline(100*y0,100*x0)/10000
    
    k1 = B0*Vy0*dt
    m1 = -B0*Vx0*dt
    k2 = B0*(Vy0+m1/2)*dt
    m2 = -B0*(Vx0+k1/2)*dt
    k3 = B0*(Vy0+m2/2)*dt
    m3 = -B0*(Vx0+k2/2)*dt
    k4 = B0*(Vy0+m3)*dt
    m4 = -B0*(Vx0+k3)*dt
    
#     k1 = B0*Vy0*dt
#     m1 = -B0*Vx0*dt
#     x2 = x0 + (Vx0+k1/2)*dt
#     y2 = y0 + (Vy0+m1/2)*dt
#     B2 = spline(100*y2,100*x2)*const
#     k2 = B2*(Vy0+m1/2)*dt
#     m2 = -B2*(Vx0+k1/2)*dt
#     x3 = x0 + (Vx0+k2/2)*dt
#     y3 = y0 + (Vy0+m2/2)*dt
#     B3 = spline(100*y3,100*x3)*const
#     k3 = B3*(Vy0+m2/2)*dt
#     m3 = -B3*(Vx0+k2/2)*dt
#     x4 = x0 + (Vx0+k3)*dt
#     y4 = y0 + (Vy0+m3)*dt
#     B4 = spline(100*y4,100*x4)*const
#     k4 = B4*(Vy0+m3)*dt
#     m4 = -B4*(Vx0+k3)*dt

#     k1 = B0*Vy0*dt
#     m1 = -B0*Vx0*dt
#     k2 = B0*(Vy0+m1/2)*dt
#     m2 = -B0*(Vx0+k1/2)*dt
#     Vx1 = Vx0 + k2
#     Vy1 = Vy0 + m2
    
    Vx1 = Vx0 + (k1+2*k2+2*k3+k4)/6
    Vy1 = Vy0 + (m1+2*m2+2*m3+m4)/6
    
    return x1,y1,Vx1,Vy1,B1
    
    

In [ ]:
## Solving an equation of motion in magnetic field
alpha = 0
stop = 0
while abs(alpha_req-alpha)>0.0001:
#while stop<1:
    ## Positron
    x1=0
    y1=0
    Vx1=Vx0
    Vy1=Vy0
    B1 = B
    q=e
    coordX1 = [x1]
    coordY1 = [y1]
    coordVx1 = [Vx1]
    coordVy1 = [Vy1]
    n_points=2
    decompB1 = np.zeros((2*n_points+1,30000))
    points1 = [i for i in range(-n_points,n_points+1)]
    print(points1)
    #orbit = [0]
    d = 0.01
    coordB1 = [B1]
    k = 0
    
    while x1<=0.84 and x1>=-0.84 and y1<=0.03 and y1>=-0.09:
        for i in range(0,2*n_points+1):
            x2 = x1 + abs(Vy1/V_full)*d*points1[i]
            y2 = y1 + abs(Vx1/V_full)*d*points1[i]
            
            decompB1[i][k] = spline(100*y2,100*x2)/10000 - B1
       
        
        x1,y1,Vx1,Vy1,B1 = rungeKutta(x1,y1,Vx1,Vy1,q)
        coordB1.append(B1)
        
        coordX1.append(x1)
        coordY1.append(y1)
        coordVx1.append(Vx1)
        coordVy1.append(Vy1)
       
        
        
        k = k + 1
    print(k)

    ## Electron
    x1=0
    y1=0
    Vx1=-Vx0
    Vy1=Vy0
    B1 = B
    q=-e
    coordX2 = [x1]
    coordY2 = [y1]
    coordVx2 = [Vx1]
    coordVy2 = [Vy1]
    decompX2 = np.zeros((2*n_points+1,30000))
    decompY2 = np.zeros((2*n_points+1,30000))
    decompB2 = np.zeros((2*n_points+1,30000))
    points2 = [i for i in range(n_points,-n_points-1,-1)]
    coordB2 = [B1]
    k = 0
    while x1<=0.84 and x1>=-0.84 and y1<=0.03 and y1>=-0.09:
        for i in range(0,2*n_points+1):
            x2 = x1 + abs(Vy1/V_full)*d*points1[i]
            y2 = y1 + abs(Vx1/V_full)*d*points2[i]
            decompB2[i][k] = spline(100*y2,100*x2)/10000 - B1
          
        
        x1,y1,Vx1,Vy1,B1 = rungeKutta(x1,y1,Vx1,Vy1,q)
        coordB2.append(B1)
        
        coordX2.append(x1)
        coordY2.append(y1)
        coordVx2.append(Vx1)
        coordVy2.append(Vy1)
        
        
        
        k = k + 1
    print(k)
    stop = 1
    alpha_1 = np.arctan(abs(coordVy1[len(coordVy1)-1]/coordVx1[len(coordVy1)-1]))*360/(2*np.pi)
    alpha_2 = np.arctan(abs(coordVy2[len(coordVy2)-1]/coordVx2[len(coordVy2)-1]))*360/(2*np.pi) 
    alpha = alpha_1+alpha_2
    E = E*alpha/alpha_req
    gamma = E/mc # Lorentz factor 
    Vx0 = (1-1/gamma**2)**(1/2)*c
    V_full=(1-1/gamma**2)**(1/2)*c
    print(alpha)
    print("Energy:", E/10E8, "GeV")
    
print(E)

In [ ]:
## Vizualization of particles tracking
print("Energy:", E/10E8, "GeV")
print(E)
alpha_1 = np.arctan(abs(coordVy1[len(coordVy1)-1]/coordVx1[len(coordVy1)-1]))*360/(2*np.pi)
alpha_2 = np.arctan(abs(coordVy2[len(coordVy2)-1]/coordVx2[len(coordVy2)-1]))*360/(2*np.pi)
print("Positron angle: ", alpha_1)
print("Electron angle: ", alpha_2)
print("Bending angle: ", alpha_1+alpha_2)

import plotly.express as px

fig = go.Figure(go.Scatter(x=[-0.84,-0.84,0.84,0.84], y=[-0.9,0.3,0.3,-0.9], fill="toself"))
fig.add_trace(go.Scatter(x=coordX1, y=coordY1,  name='Positron',line=dict(color='Blue')))
fig.update_layout(legend_orientation="h",autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))

fig.add_trace(go.Scatter(x=coordX2, y=coordY2,  name='Electron',line=dict(color='Red')))
fig.update_layout(legend_orientation="h",autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()



In [ ]:
## Field decomposition into harmonics: B, K, S, O
from scipy.optimize import curve_fit
from scipy.optimize import lsq_linear


    
def func(y,G,S,O):
    return G*y+(S/2)*y**2+(O/6)*y**3


dataY1 = [d*i for i in range(-n_points,n_points+1)]
dataY2 = [d*i for i in range(n_points,-n_points-1,-1)]
#Matrix = np.zeros((k,2))
quad1=[]
sext1=[]
oct1=[]
quad2=[]
sext2=[]
oct2=[]
orbit1=[]
orbit2=[]
s1=0
s2=0
integralB=0
integralG=0
integralS=0
integralO=0
for i in range(0,k-1):
    popt1,pcov1 = curve_fit(func,dataY1,decompB1[:,i])
    popt2,pcov2 = curve_fit(func,dataY2,decompB2[:,i])
    quad1.append(popt1[0])
    sext1.append(popt1[1])
    oct1.append(popt1[2])
    quad2.append(popt2[0])
    sext2.append(popt2[1])
    oct2.append(popt2[2])
    orbit1.append(s1)
    orbit2.append(s2)
    ds1=((coordX1[i]-coordX1[i+1])**2+(coordY1[i]-coordY1[i+1])**2)**(0.5)
    ds2=((coordX2[i]-coordX2[i+1])**2+(coordY2[i]-coordY2[i+1])**2)**(0.5)
    s1=s1+ds1
    s2=s2-ds2
    integralB=integralB+ds1*coordB1[i]+ds2*coordB2[i]
    integralG=integralG+ds1*popt1[0]+ds2*popt2[0]
    integralS=integralS+ds1*popt1[1]+ds2*popt2[1]
    integralO=integralO+ds1*popt1[2]+ds2*popt2[2]
    
popt1,pcov1 = curve_fit(func,dataY1,decompB1[:,0])
G0=popt1[0]
S0=popt1[1]
O0=popt1[2]
interB1=[func(dataY1[i],G0,S0,O0) for i in range(0,5)]

print("B0 [T]:",11294.59861114771/10000)
print("G0 [T/m]:", G0)
print("S0 [T/m^2]:", S0)
print("O0 [T/m^3]:", O0)
print("L_eff [m]:",integralB/11294.59861114771)
print("B_int [T*m]:",integralB)
print("G_int [T]:",integralG)
print("S_int [T/m]:",integralS)
print("O_int [T/m^2]:",integralO)

coordB2.pop()
quad2.reverse()
sext2.reverse()
oct2.reverse()
coordB2.reverse()
orbit2.reverse()
coordB1.pop()

dipole=coordB2+coordB1
quad=quad2+quad1
sext=sext2+sext1
oct=oct2+oct1
orbit=orbit2+orbit1


fig = go.Figure()
fig.add_trace(go.Scatter(x=dataY1, y=decompB1[:,0],  name='Positron'))
fig.add_trace(go.Scatter(x=dataY1, y=interB1,  name='Positron',line=dict(color='Red')))
fig.update_layout(title = "Dipole strength",legend_orientation="h",autosize=False, xaxis=dict(title="s,m",dtick=0.2,tickangle=0),
                  yaxis=dict(title="K0,T"),
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
## Visualization of Dipole, Quadrupole, Sextupole and Octupole harmonics
fig = go.Figure()
fig.add_trace(go.Scatter(x=orbit, y=dipole,  name='Positron',line=dict(color='Blue')))
fig.update_layout(title = "Dipole strength",legend_orientation="h",autosize=False, xaxis=dict(title="s,m",dtick=0.2,tickangle=0),
                  yaxis=dict(title="K0,T"),
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()
                  
fig = go.Figure()
fig.add_trace(go.Scatter(x=orbit, y=quad,  name='Positron',line=dict(color='Blue')))
fig.update_layout(title = "Gradient strength",legend_orientation="h",autosize=False, xaxis=dict(title="s,m",dtick=0.2,tickangle=0),
                  yaxis=dict(title="K1,T/m"),
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=orbit, y=sext,  name='Positron',line=dict(color='Blue')))
fig.update_layout(title = "Sextupole strength",legend_orientation="h",autosize=False, xaxis=dict(title="s,m",dtick=0.2,tickangle=0),
                  yaxis=dict(title="K2,T/m^2"),
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=orbit, y=oct,  name='Positron',line=dict(color='Blue')))
fig.update_layout(title = "Octupole strength",legend_orientation="h",autosize=False, xaxis=dict(title="s,m",dtick=0.2,tickangle=0),
                  yaxis=dict(title="K3,T/m^3"),
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()


In [ ]:
a=15
b=30000
print(a/b*360/(2*np.pi))
print(np.arctan(a/b)*360/(2*np.pi))